In [2]:
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "notebook"


import locale
locale.setlocale(locale.LC_NUMERIC, 'fr_FR')

engine = create_engine("sqlite:///c:/Users/antoine.herman/Desktop/indicateurs_tdc.sqlite3")

In [2]:
comm_litt = pd.read_csv("C:/Users/antoine.herman/Desktop/communes_litt_mer_cog2021.csv")
comm_litt

,idcom,idcomtxt,iddep
0,97101,ABYMES,971
1,97104,BAILLIF,971
2,97102,ANSE-BERTRAND,971
3,97105,BASSE-TERRE,971
4,97103,BAIE-MAHAULT,971
...,...,...,...
957,85278,SAINT-VINCENT-SUR-JARD,085
958,85288,TALMONT-SAINT-HILAIRE,085
959,85294,LA TRANCHE-SUR-MER,085
960,85297,TRIAIZE,085


In [36]:
national = pd.read_sql("SELECT * FROM indicateurs_national;", con=engine)
national["seuil_frange"] = national["seuil_frange"].astype(str)
national["lbl_frange"] = national["seuil_frange"].apply(lambda x: x + " mètres")

national["p_urba"] = national["surfaces_naf"] / (national["surfaces_naf"] + national["surfaces_urba"])
national["surfaces_naf"] = national["surfaces_naf"] / 10000000
national["surfaces_urba"] = national["surfaces_urba"] / 10000000
national["nb_logt"] = national["nb_logt"] / 1000
national["nb_logt_rs"] = national["nb_logt_rs"] / 1000
national["nb_logt_po"] = national["nb_logt_po"] / 1000
national["nb_logt_pb"] = national["nb_logt_pb"] / 1000
national["nb_logt_va"] = national["nb_logt_va"] / 1000
national["nb_loc_act"] = national["nb_loc_act"] / 1000
national["nb_maisons"] = national["nb_maisons"] / 1000
national["nb_appts"] = national["nb_appts"] / 1000
national["tx_maison"] = national["nb_maisons"] / national["nb_logt"] * 100.0
national["tx_appt"] = national["nb_appts"] / national["nb_logt"] * 100.0

national["nb_commerces"]  = national["nb_commerces"] / 1000
national["nb_bureaux"]  = national["nb_bureaux"] / 1000
national["nb_campings"]  = national["nb_campings"] / 1000
national["nb_hotels"]  = national["nb_hotels"] / 1000
national["nb_act_autres"]  = national["nb_act_autres"] / 1000

national["tx_commerces"]  = national["nb_commerces"] / national["nb_loc_act"] * 100.0  
national["tx_bureaux"]  = national["nb_bureaux"] / national["nb_loc_act"] * 100.0
national["tx_camping"]  = national["nb_campings"] / national["nb_loc_act"] * 100.0
national["tx_hotels"]  = national["nb_hotels"] / national["nb_loc_act"] * 100.0
national["tx_act_autres"]  = national["nb_act_autres"] / national["nb_loc_act"] * 100.0
national 

,index,zone,seuil_frange,nb_locaux,nb_logt,estim_logt,estim_loyer,nb_loc_act,estim_bur_com,nb_logt_po,...,surfaces_habitables,lbl_frange,p_urba,tx_maison,tx_appt,tx_commerces,tx_bureaux,tx_camping,tx_hotels,tx_act_autres
0,0,Littoral,200,1498716.0,920.977,1.968314e+11,1.421946e+08,116.005,13738857541,588.055,...,61264705.0,200 mètres,0.727592,34.973403,65.026488,48.789276,20.708590,0.603422,3.857592,26.066118
1,1,Littoral,1000,4821083.0,3133.015,6.968145e+11,6.310567e+08,320.260,38538129784,1853.399,...,223362345.0,1000 mètres,0.752479,41.733282,58.267388,42.920127,24.328983,0.705052,2.482046,29.566914
2,2,Littoral,10000,8312551.0,5617.967,1.234510e+12,1.321157e+09,513.305,62275631686,3147.746,...,418629231.0,10000 mètres,0.847970,45.563564,54.436667,38.397444,25.014562,0.650685,1.854648,34.085193


In [37]:
color_discrete_map = {"surfaces_naf": "darkgreen", "surfaces_urba": "darkorange"}
new_names = {'surfaces_naf': 'Surface NAF', 'surfaces_urba': 'Surface Urbanisée'}

fig = px.bar(national, 
             y=["surfaces_naf", "surfaces_urba"], 
             x="lbl_frange", 
             color_discrete_map=color_discrete_map,
             text_auto=".2f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Surface (milliers d'ha)", "lbl_frange": "Bande"},
             title="Répartition des surfaces de foncier concernées en 2021",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Surface de foncier")
fig.show()

In [38]:

color_discrete_map = {"nb_logt": "indianred", "nb_loc_act": "lightsalmon"}
new_names = {'nb_logt': 'Logements', 'nb_loc_act': "Locaux d'activité"}

fig = px.bar(national, 
             y=["nb_logt", "nb_loc_act"], 
             x="lbl_frange", 
             color_discrete_map=color_discrete_map,
             text_auto=".0f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Nombre de locaux (milliers)", "lbl_frange": "Bande"},
             title="Ventilation des locaux (hors dépendance)",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Type de locaux")
fig.show()

In [243]:
df = pd.DataFrame(dict(
    value=[11, 9, 8, 8],
    bande=['200 mètres','1 000 mètres','10 000 mètres',
           'France Entière',]))

fig = px.line_polar(df, 
                    r='value', 
                    theta='bande',
                    line_close=True, 
                    width=550, 
                    height=550, 
                    title="Pourcentage de locaux d'activité selon la bande littorale",
                    )

fig.update_traces(fill='toself')
fig.update_traces(line=dict(color='lightsalmon'))


fig.show()

In [244]:
df = pd.DataFrame(dict(
    surface=[0.3, 1, 4],
    parc=[2.5, 8, 14],
    bande=['200 mètres','1 000 mètres','10 000 mètres',]))

palette = px.colors.sequential.Aggrnyl

color_discrete_map = {"surface": palette[0], "parc": palette[1]}
new_names = {'surface': 'Part de la superficie nationale', 'nb_loc_act': "Part du parc national"}

fig = px.bar(df, 
       x="bande", 
       y=["surface", "parc"], 
       color_discrete_map=color_discrete_map,
       barmode="group", 
       height=600, 
       width=700,
       labels={"value" : "Pourcentage (%)", "bande": "Bande"},
       title="Poids de la superficie et du parc de logements", )
fig.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Poids / territoire national")

In [245]:
import plotly.graph_objects as go

national["estim_tot"] = national["estim_logt"] + national["estim_bur_com"]

header=['Bande', 
        'Estimation des logements', 
        "Part de l'estimation du parc total de logements (France)", 
        "Estimation des bureaux et commerces", 
        "Estimation globale"]
values = [['200 mètres','1 000 mètres','10 000 mètres',], 
          [str(round(e/1000000000, 2)) + " Md€"  for e in national["estim_logt"].to_list()],
          ["2,5 %", "9 %", "15 %"],
          [str(round(e/1000000000, 2)) + " Md€"  for e in national["estim_bur_com"].to_list()],
          [str(round(e/1000000000, 2)) + " Md€"  for e in national["estim_tot"].to_list()],
          ]

fig = go.Figure(data=[go.Table(
  columnorder = [1,2,3,4,5],
  columnwidth = [220 ,260, 260, 260, 250],
  header =dict(values=header,
    line_color='darkslategray',
    fill_color='royalblue',
    align=["center"]*5,
    font=dict(color='white', size=16),
    height=40
),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['paleturquoise', 'white', "white", "white", "darkturquoise"]),
    align=['center',]*5,
    font_size=16,
    height=30)
    )
],
)

fig.update_layout(
    title="Estimation de la valeur vénale des parcs de logement et de locaux d'activité sur le littoral",
    width=1000,
    height=400
)
fig.show()

In [246]:
def note(seuil, annee, type):
    df = pd.read_sql_query(f"""
                            SELECT aav2020, libaav2020, valeur_ratio_{annee}_{type} AS ratio 
                            FROM indicateurs_aav 
                            WHERE seuil_frange={seuil}
                            """, 
                            con=engine, 
                            dtype={"aav2020": str})
    df["comparaison_base_100"] = df["ratio"].apply(lambda x: round(x*100, 1) if x <= 1 else round(100/(2 - x), 1))

    return df["comparaison_base_100"].mean()

df_mai = pd.DataFrame(dict(
    valorisation1=[note("200", "2015", "maison"), 
                     note("200", "2018", "maison"), 
                     note("200", "2021", "maison")],
    valorisation2=[note("1000", "2015", "maison"), 
                     note("1000", "2018", "maison"), 
                     note("1000", "2021", "maison")],
    valorisation3=[note("10000", "2015", "maison"), 
                     note("10000", "2018", "maison"), 
                     note("10000", "2021", "maison")],
    annee=['2015','2018','2021',],
    type=["Maisons moyennes (90-130 m2)"]*3
    ))

df_apt = pd.DataFrame(dict(
    valorisation1=[note("200", "2015", "appt"), 
                     note("200", "2018", "appt"), 
                     note("200", "2021", "appt")],
    valorisation2=[note("1000", "2015", "appt"), 
                     note("1000", "2018", "appt"), 
                     note("1000", "2021", "appt")],
    valorisation3=[note("10000", "2015", "appt"), 
                     note("10000", "2018", "appt"), 
                     note("10000", "2021", "appt")],
    type=['Appartements 3/4 pièces']*3,
    annee=['2015','2018','2021',]))

df = pd.concat([df_mai, df_apt])

palette = px.colors.qualitative.Prism

color_discrete_map = {"valorisation1": palette[0], 
                      "valorisation2": palette[1],
                      "valorisation3": palette[2],
                      }

new_names = {"valorisation1": "200 mètres", 
            "valorisation2": "1 000 mètres",
            "valorisation3": "10 000 mètres",
            }

fig = px.line(df, 
       x="annee", 
       y=["valorisation1", "valorisation2", "valorisation3"], 
       color_discrete_map=color_discrete_map,
       height=550, 
       width=1000,
       facet_col="type",
       markers=True,
       labels={"value" : "Base 100", "annee": "Années", "type": "Type de Logements"},
       title="Valorisation moyenne des logements impactés dans leur aire de marché (AAV)", )

fig.update_yaxes(range=[98, 130])
# Mettre à jour les noms de traces
for trace in fig.data:
    trace.name = new_names[trace.name]
fig.update_layout(legend_title_text="Bandes")

In [8]:
national["part_rs"] = national["nb_logt_rs"] / national["nb_logt"] * 100
print(national["part_rs"])
df = pd.DataFrame(dict(
    value=national["part_rs"].to_list() + [9.5,],
    bande=['200 mètres','1 000 mètres','10 000 mètres',
           'France Entière',]))

fig = px.line_polar(df, 
                    r='value', 
                    theta='bande',
                    line_close=True, 
                    width=550, 
                    height=550, 
                    title="Part de résidences secondaires selon la bande littorale (%)",
                    )

fig.update_traces(fill='toself')
fig.update_traces(line=dict(color='indianred'))


fig.show()

0    33.209841
1    23.750158
2    16.403639
Name: part_rs, dtype: float64


In [9]:
national['nb_logt_rp'] = national['nb_logt_po'] - national['nb_logt_rs'] #- national['nb_logt_va']
color_discrete_map = {"nb_logt_rp": "indianred", "nb_logt_rs": "lightsalmon"}
new_names = {'nb_logt_rp': 'Résidences principales', 'nb_logt_rs': "Résidences secondaires"}

fig = px.bar(national, 
             y=["nb_logt_rp", "nb_logt_rs"], 
             x="lbl_frange", 
             color_discrete_map=color_discrete_map,
             text_auto=".0f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Nombre de logements (milliers)", "lbl_frange": "Bande"},
             title="Résidences principales et secondaires sur le littoral",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Type de locaux")
fig.show()

In [52]:
df = pd.read_sql_query(f"""
    SELECT iddep AS "Département", libcom AS "Commune", cast(nb_logt_rs AS integer) AS "Résidences secondaires"
    FROM indicateurs_com_200m
    ORDER BY nb_logt_rs DESC
    LIMIT 20
    """, 
    con=engine, 
    )

header=['Dép.',
        'Communes', 
        'Résidences secondaires', 
        ]
values = [df["Département"].to_list(),df["Commune"].to_list(), df["Résidences secondaires"].to_list(), 
          ]

fig = go.Figure(data=[go.Table(
  columnorder = [1,2,3],
  columnwidth = [100, 300 ,200,],
  header =dict(values=header,
    line_color='darkslategray',
    fill_color='royalblue',
    align=["center", "left","center"],
    font=dict(color='white', size=14),
    height=40
),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['lightsalmon','lightsalmon', 'white',]),
    align=["center","left",'center',],
    font_size=14,
    height=25)
    )
],
)

fig.update_layout(
    #title="Communes ayant le + de résidences secondaires dans la bande des 200 m",
    width=700,
    height=740
)
fig.show()

In [53]:
df = pd.read_sql_query(f"""
    SELECT iddep AS "Département",  libcom AS "Commune", round(nb_logt_rs*100.0/nb_logt, 2) AS taux
    FROM indicateurs_com_200m
    WHERE nb_logt > 10
    ORDER BY taux DESC
    LIMIT 20
    """, 
    con=engine, 
    )

header=['Dép.',
        'Communes', 
        'Résidences secondaires', 
        ]
values = [df['Département'] ,df["Commune"].to_list(), df["taux"].to_list(), 
          ]

fig = go.Figure(data=[go.Table(
  columnorder = [1,2,3],
  columnwidth = [100, 300 ,200,],
  header =dict(values=header,
    line_color='darkslategray',
    fill_color='royalblue',
    align=['center',"left","center"],
    font=dict(color='white', size=14),
    height=40
),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['lightsalmon', 'lightsalmon', 'white',]),
    align=['center',"left",'center',],
    font_size=14,
    height=25)
    )
],
)

fig.update_layout(
    #title="Communes au taux de résidences secondaires élevé (bande 200 m)",
    width=700,
    height=740
)
fig.show()

In [255]:
color_discrete_map = {"nb_logt_po": "indianred", "nb_logt_pb": "lightsalmon", "nb_logt_va": "darkorange"}
new_names = {'nb_logt_po': 'Propriétaire occupant (RP+RS)', 'nb_logt_pb': "Propriétaire bailleur", "nb_logt_va": "Vacant"}

fig = px.bar(national, 
             y=["nb_logt_po", "nb_logt_pb", "nb_logt_va"], 
             x="lbl_frange", 
             color_discrete_map=color_discrete_map,
             text_auto=".0f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Nombre de logements (milliers)", "lbl_frange": "Bande"},
             title="Mode d'occupation des logements sur le littoral",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Type d'occupation")
fig.show()

In [266]:
#color_discrete_map = {"nb_maisons": "indianred", "nb_appts": "lightsalmon"}
new_names = {'nb_maisons': 'Maisons', 'nb_appts': "Appartements"}

fig = px.bar(national, 
             y=["nb_maisons", "nb_appts"], 
             x="lbl_frange", 
             #color_discrete_map=color_discrete_map,
             text_auto=".0f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Nombre de logements (milliers)", "lbl_frange": "Bande"},
             title="Forme des logements sur le littoral",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Type de locaux")
fig.show()

In [269]:
categories =['200 mètres','1 000 mètres','10 000 mètres',
           'France Entière',]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=national["tx_maison"].to_list() + [53,],
      theta=categories,
      fill='toself',
      name='Part de maisons',
))
fig.add_trace(go.Scatterpolar(
      r=national["tx_appt"].to_list() + [47,],
      theta=categories,
      fill='toself',
      name="Part d'appartements"
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
    )),
  showlegend=True,
  title="Part des maisons et appartements selon la bande littorale (%)",
)

fig.show()



In [15]:
y

In [17]:
def tx(seuil, periode):
     df = pd.read_sql_query(f"""select round(nb_maisons_{periode} * 100 / (nb_maisons_av45 + nb_maisons_45_59 + nb_maisons_60_74 + nb_maisons_75_97 + nb_maisons_98_12 + nb_maisons_ap12 ),1) as tx 
                        from indicateurs_national
                       WHERE seuil_frange={seuil};""", con=engine)
     return df["tx"].values[0]
     

top_labels = ['Avant 1945', '1945-1959', '1960-1974', '1975-1997', "1998-2012",
              'Après 2012']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)', 'rgba(200, 200, 220, 1)']

x_data = [[27, 10, 17, 26, 12,  8],
          [tx("10000", "av45"), 
           tx("10000", "45_59"), 
           tx("10000", "60_74"), 
           tx("10000", "75_97"), 
           tx("10000", "98_12"), 
           tx("10000", "ap12")],
          [tx("1000", "av45"), 
           tx("1000", "45_59"), 
           tx("1000", "60_74"), 
           tx("1000", "75_97"), 
           tx("1000", "98_12"), 
           tx("1000", "ap12")],
          [tx("200", "av45"), 
           tx("200", "45_59"), 
           tx("200", "60_74"), 
           tx("200", "75_97"), 
           tx("200", "98_12"), 
           tx("200", "ap12")],
          ]

y_data = ['France entière',
          '10 000 mètres', '1 000 mètres',
          '200 mètres']

fig = go.Figure()

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Bar(
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(l=120, r=10, t=140, b=80),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd,
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

fig.update_layout(annotations=annotations, title="Ventilation des maisons selon la période de construction", width=1150)

fig.show()

In [56]:
def tx(seuil, periode):
     df = pd.read_sql_query(f"""select round(nb_appts_{periode} * 100 / (nb_appts_petits + nb_appts_moyens + nb_appts_grands),1) as tx 
                        from indicateurs_national
                       WHERE seuil_frange={seuil};""", con=engine)
     return df["tx"].values[0]
     

top_labels = ['Petite', 'Moyens', 'Grands']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)', 'rgba(200, 200, 220, 1)']

x_data = [[41, 38, 21],
          [tx("10000", "petits"), 
           tx("10000", "moyens"), 
           tx("10000", "grands"), 
           ],
          [tx("1000", "petits"), 
           tx("1000", "moyens"), 
           tx("1000", "grands"), 
           ],
          [tx("200", "petits"), 
           tx("200", "moyens"), 
           tx("200", "grands"), 
           ],
          ]

y_data = ['France entière',
          '10 000 mètres', '1 000 mètres',
          '200 mètres']

fig = go.Figure()

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Bar(
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(l=120, r=10, t=140, b=80),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd,
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

fig.update_layout(annotations=annotations, title="Ventilation des appartements selon la taille", width=1150)

fig.show()

In [ ]:
def tx(seuil, periode):
     df = pd.read_sql_query(f"""select round(nb_maisons_{periode} * 100 / (nb_maisons_petites + nb_maisons_moyennes + nb_maisons_grandes),1) as tx 
                        from indicateurs_national
                       WHERE seuil_frange={seuil};""", con=engine)
     return df["tx"].values[0]
     

top_labels = ['Petites', 'Moyennes', 'Grandes']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)', 'rgba(200, 200, 220, 1)']

x_data = [[41, 38, 21],
          [tx("10000", "petites"), 
           tx("10000", "moyennes"), 
           tx("10000", "grandes"), 
           ],
          [tx("1000", "petites"), 
           tx("1000", "moyennes"), 
           tx("1000", "grandes"), 
           ],
          [tx("200", "petites"), 
           tx("200", "moyennes"), 
           tx("200", "grandes"), 
           ],
          ]

y_data = ['France entière',
          '10 000 mètres', '1 000 mètres',
          '200 mètres']

fig = go.Figure()

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Bar(
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(l=120, r=10, t=140, b=80),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd,
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

fig.update_layout(annotations=annotations, title="Ventilation des maisons selon la taille", width=1150)

fig.show()

In [31]:
categories =['200 mètres','1 000 mètres','10 000 mètres',
           'France Entière',]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=national["tx_bureaux"].to_list() + [26,],
      theta=categories,
      name='Bureaux',
))
fig.add_trace(go.Scatterpolar(
      r=national["tx_commerces"].to_list() + [30,],
      theta=categories,
      name="Commerces"
))
fig.add_trace(go.Scatterpolar(
      r=national["tx_hotels"].to_list() + [1,],
      theta=categories,
      name="Hotels"
))
fig.add_trace(go.Scatterpolar(
      r=national["tx_act_autres"].to_list() + [41,],
      theta=categories,
      name="Autres"
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
    )),
  showlegend=True,
  title="Repartition des locaux d'activités selon la bande (%)",
  width=700,
  height=600
)

fig.show()


In [47]:
palette = px.colors.qualitative.Prism
new_names = {'nb_commerces': 'Commerces', 'nb_bureaux': "Bureaux", "nb_loc_autres": "Autres", "nb_hotels": "Hotels", "nb_campings": "Campings"}
color_discrete_map = {}
for i, c in enumerate(new_names.keys()):
    color_discrete_map[c] = palette[i]

fig = px.bar(national, 
             y=["nb_commerces", "nb_bureaux", "nb_act_autres", "nb_hotels", "nb_campings"], 
             x="lbl_frange", 
             color_discrete_map=color_discrete_map,
             text_auto=".0f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Nombre de locaux (milliers)", "lbl_frange": "Bande"},
             title="Ventilation des locaux (hors dépendance)",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Type d'activité", width=700, height=700)
fig.show()

In [66]:
df = national[["lbl_frange", "estim_loyer"]]
df["estim_loyer"] = df["estim_loyer"].apply(lambda x : round(x / 1000000, 2))

header=['Bande',
        "Estimation des loyers mensuels perçus (en million d'€/mois)", 
        ]
values = [df["lbl_frange"].to_list(),df["estim_loyer"].to_list() 
          ]

fig = go.Figure(data=[go.Table(
  columnorder = [1,2],
  columnwidth = [100, 300],
  header =dict(values=header,
    line_color='darkslategray',
    fill_color='royalblue',
    align=["left", "center",],
    font=dict(color='white', size=14),
    height=40
),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['lightsalmon','white']),
    align=["left","center",],
    font_size=14,
    height=25)£
    )
],
)

fig.update_layout(
    #title="Communes ayant le + de résidences secondaires dans la bande des 200 m",
    width=700,
    height=350
)
fig.show()

C:\Users\antoine.herman\AppData\Local\Temp\ipykernel_13056\3910548764.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

